In [ ]:
"""
type: 입력데이터종류
0 - (string) 계좌번호
1 - (string) 상품관리구분코드
2 - (long) 요청건수[default:14] - 최대 50개
3 - (string) 수익률구분코드 - ( "1" : 100% 기준, "2": 0% 기준)
value: 새로지정할값
value = object.GetHeaderValue(type)
type에해당하는헤더데이터를반환합니다
type: 데이터종류
0 - (string) 계좌명
1 - (long) 결제잔고수량
2 - (long)체결잔고수량
3 - (longlong)평가금액(단위:원)
4 - (longlong)평가손익(단위:원)
5 - 사용하지않음
6 - (longlong)대출금액(단위:원)
7 - (long) 수신개수
8 - (double) 수익율
9 - (longlong) D+2 예상예수금
10 - (longlong) 대주평가금액
11 - (longlong) 잔고평가금액
12 - (longlong) 대주금액

반환값: 데이터종류에해당하는값
value = object.GetDataValue(Type,Index)
type에해당하는데이터를반환합니다
type: 데이터종류

0 - (string) 종목명
1 - (char)신용구분
    코드
    내용
    'Y'
    'D'
    'B'
    'M'
    'P'
    'I'
    신용융자/유통융자
    신용대주/유통대주
    담보대출
    매입담보대출
    플러스론대출
    자기융자/유통융자
2 - (string) 대출일
3 - (long)결제잔고수량
4 - (long)결제장부단가
5 - (long)전일체결수량
6 - (long)금일체결수량
7 - (long)체결잔고수량
9 - (longlong)평가금액(단위:원) - 천원미만은내림
10 - (longlong)평가손익(단위:원) - 천원미만은내림
11 - (double)수익률
12 - (string) 종목코드
13 - (char)주문구분
15 - (long)매도가능수량
16 - (string) 만기일
17 - (double) 체결장부단가
18 - (longlong) 손익단가
반환값: 데이터종류의 index번째 data
object.BlockRequest()
계좌별잔고및주문체결평가현황관련데이터요청.Blocking Mode
"""

In [ ]:
import sys
from PyQt5.QtWidgets import *
import win32com.client

In [ ]:
# 설명: 주식 계좌잔고 종목(최대 200개)을 가져와 현재가  실시간 조회하는 샘플
# CpEvent: 실시간 현재가 수신 클래스
# CpStockCur : 현재가 실시간 통신 클래스
# Cp6033 : 주식 잔고 조회
# CpMarketEye: 복수 종목 조회 서비스 - 200 종목 현재가를 조회 함.

In [ ]:
# Cp6033 : 주식 잔고 조회
class Cp6033:
    def __init__(self):
        # 통신 OBJECT 기본 세팅
        self.objTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")
        initCheck = self.objTrade.TradeInit(0)
        if (initCheck != 0):
            print("주문 초기화 실패")
            return
 
        acc = self.objTrade.AccountNumber[0]  # 계좌번호
        accFlag = self.objTrade.GoodsList(acc, 1)  # 주식상품 구분
        print(acc, accFlag[0])
 
        self.objRq = win32com.client.Dispatch("CpTrade.CpTd6033")
        self.objRq.SetInputValue(0, acc)  # 계좌번호
        self.objRq.SetInputValue(1, accFlag[0])  # 상품구분 - 주식 상품 중 첫번째
        self.objRq.SetInputValue(2, 50)  #  요청 건수(최대 50)
 
    # 실제적인 6033 통신 처리
    def rq6033(self, retcode):
        self.objRq.BlockRequest()
 
        # 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        rqRet = self.objRq.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
     
    
        # 종목 개수
        vrrd = self.objRq.GetHeaderValue(3)
        vrtd = self.objRq.GetHeaderValue(4)
        cnt = self.objRq.GetHeaderValue(7)
        tdf = self.objRq.GetHeaderValue(8)
        
        print("평가금액 : {} \t 평가손익 : {} \t 수익률 : {}".format(vrrd, vrtd, tdf))
    
        asdf = []
         
        for i in range(cnt):
        
            name = self.objRq.GetDataValue(0, i)  # 종목명
            cashFlag = self.objRq.GetDataValue(1, i)  # 신용구분
            date = self.objRq.GetDataValue(2, i)  # 대출일
            amount = self.objRq.GetDataValue(7, i) # 체결잔고수량
            evalValue = self.objRq.GetDataValue(9, i) # 평가금액(천원미만은 절사 됨)
            evalPerc = self.objRq.GetDataValue(11, i) # 평가손익
            code = self.objRq.GetDataValue(12, i)  # 종목코드
            buyPrice = self.objRq.GetDataValue(17, i) # 체결장부단가
            
            asdf.append([code, name, cashFlag, amount, buyPrice, evalValue, evalPerc])
            retcode.append(code)
    
        return asdf
 
    def Request(self, retCode):
        
        return self.rq6033(retCode)

In [ ]:
codes = []
obj6033 = Cp6033()
tmp = obj6033.Request(codes)

In [ ]:
import pandas as pd

In [ ]:
DF = pd.DataFrame(tmp)
DF.columns = "종목코드 종목명 신용구분 체결잔고수량 체결장부단가 평가금액 평가손익".split(" ")

In [ ]:
DF